#📌 Extracão

1. Verificação Inicial dos Dados

1.1 Carregar os Dados

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/mlpoliveira/challenge2-data-science/main/TelecomX_Data.json'
df = pd.read_json(url)

# Visualizar as primeiras 5 linhas
print(df.head())

1.2 Verificar Estrutura do DataFrame

In [2]:
print("\nFormato do DataFrame (linhas, colunas):")
print(df.shape)  # Saída esperada: (7267, 6)

print("\nColunas disponíveis:")
print(df.columns.tolist())  # ['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']


Formato do DataFrame (linhas, colunas):
(7267, 6)

Colunas disponíveis:
['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']


1.3 Verificar Tipos de Dados


In [3]:
print("\nTipos de dados de cada coluna:")
print(df.dtypes)


Tipos de dados de cada coluna:
customerID    object
Churn         object
customer      object
phone         object
internet      object
account       object
dtype: object


2. Verificação de Dados Ausentes

2.1 Verificar Valores Nulos Explícitos


In [4]:
print("\nValores nulos por coluna:")
print(df.isnull().sum())


Valores nulos por coluna:
customerID    0
Churn         0
customer      0
phone         0
internet      0
account       0
dtype: int64


2.2 Verificar Valores Vazios ou Inconsistentes


In [5]:
print("\nValores vazios ou 'unknown':")
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"{col}: {df[col].str.contains('unknown|na|none', case=False, na=False).sum()}")


Valores vazios ou 'unknown':
customerID: 44
Churn: 0
customer: 0
phone: 0
internet: 0
account: 0


3. Análise Inicial das Variáveis


3.1 Análise da Variável Target (Churn)


In [6]:
print("\nDistribuição de Churn (evasão):")
print(df['Churn'].value_counts(normalize=True) * 100)


Distribuição de Churn (evasão):
Churn
No     71.198569
Yes    25.719004
        3.082427
Name: proportion, dtype: float64


3.2 Verificar Dados Aninhados (JSON)


In [7]:
import json

# Visualizar a estrutura de uma linha (exemplo: primeira linha)
print("\nExemplo de dados aninhados em 'customer':")
print(json.dumps(df['customer'].iloc[0], indent=2))


Exemplo de dados aninhados em 'customer':
{
  "gender": "Female",
  "SeniorCitizen": 0,
  "Partner": "Yes",
  "Dependents": "Yes",
  "tenure": 9
}


4. Extração de Dados Aninhados (Pré-Transformação)


4.1 Verificar Chaves Únicas em Cada Coluna JSON


In [8]:
# Extrair todas as chaves únicas nos dicionários
def get_unique_keys(series):
    keys = set()
    for item in series:
        if isinstance(item, dict):
            keys.update(item.keys())
    return keys

print("\nChaves únicas em 'customer':", get_unique_keys(df['customer']))
print("Chaves únicas em 'phone':", get_unique_keys(df['phone']))
print("Chaves únicas em 'internet':", get_unique_keys(df['internet']))
print("Chaves únicas em 'account':", get_unique_keys(df['account']))


Chaves únicas em 'customer': {'Dependents', 'Partner', 'gender', 'tenure', 'SeniorCitizen'}
Chaves únicas em 'phone': {'PhoneService', 'MultipleLines'}
Chaves únicas em 'internet': {'StreamingMovies', 'OnlineSecurity', 'InternetService', 'StreamingTV', 'TechSupport', 'OnlineBackup', 'DeviceProtection'}
Chaves únicas em 'account': {'Contract', 'PaperlessBilling', 'Charges', 'PaymentMethod'}


4.2 Verificar Valores Únicos em Campos Críticos


In [9]:
# Exemplo: Verificar valores únicos em 'Contract' (dentro de 'account')
contract_values = set()
for item in df['account']:
    if isinstance(item, dict) and 'Contract' in item:
        contract_values.add(item['Contract'])
print("\nValores únicos em 'Contract':", contract_values)


Valores únicos em 'Contract': {'One year', 'Two year', 'Month-to-month'}


In [10]:
# backup
df.to_csv('dados_brutos.csv', index=False)

#🔧 Transformação

In [19]:
# Copiando o DataFrame original para não mexer diretamente nele
df_limpo = df.copy()

In [20]:
# Expandir a coluna "customer"
customer_df = pd.json_normalize(df_limpo['customer'])
phone_df = pd.json_normalize(df_limpo['phone'])
internet_df = pd.json_normalize(df_limpo['internet'])
account_df = pd.json_normalize(df_limpo['account'])

# Juntar tudo em um único DataFrame
df_final = pd.concat([df_limpo[['customerID', 'Churn']], customer_df, phone_df, internet_df, account_df], axis=1)

# Exibir as primeiras linhas para verificar
df_final.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [22]:
df_final.isnull().sum()
df_final.dtypes



customerID           object
Churn                object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
Charges.Monthly     float64
Charges.Total        object
dtype: object

#📊 Carga e análise

In [25]:
df_final.to_csv('dados_limpos.csv', index=False)

#📄Relatorio Final